In [1]:
import os
from dotenv import load_dotenv

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tools import Tool
from langchain.agents import create_agent

load_dotenv()


True

In [2]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tools import Tool
# Custom tool
def my_tool_function(query: str) -> str:
    return f"Tool response: {query}"

my_tool = Tool.from_function(
    func=my_tool_function,
    name="simple_tool",
    description="Use this for simple responses."
)

# DuckDuckGo search tool
ddg_search = DuckDuckGoSearchRun()

tools = [ddg_search, my_tool]


In [3]:
import os
from dotenv import load_dotenv

from langchain_google_genai import ChatGoogleGenerativeAI

llm_agent1 = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0
)
agent1 = create_agent(
    llm_agent1,
    tools=tools,
    system_prompt=(
        "You are Agent 1: The Recipe Scout. "
        "Research and list items clearly and concisely."
    )
)
response1 = agent1.invoke({
    "messages": [
        ("user", "List 5 recipes good for blood sugar stability.")
    ]
})

agent1_output = response1["messages"][-1].content

print("===== Agent 1 Output =====")
print(agent1_output)


===== Agent 1 Output =====
[{'type': 'text', 'text': 'Here are 5 recipes good for blood sugar stability:\n\n1.  Turkey Chili\n2.  Turkey and Quinoa Bowls with Roasted Vegetables and Avocado\n3.  Oat Porridge with Berries and Nuts\n4.  Vegetable and Lean Protein Fritters\n5.  Green Smoothie with Protein Powder and Nut Butter', 'extras': {'signature': 'CtMLAXLI2nwRkVkQfVMkRVJru8O4EG6WouK5iVryRJrU5UzCaXGlqBoMv/23+D8xpQPafV5wi9HCVTmPwlC4STYl+NR8/7xFI5ccddbpaK8wpfGhvs7fhwE9QZ6QNYiVOr2OTI9JPXIBvzHvV68MAXRtDcGGNck5T5/pHIW+d12feARn0TvMAgWVpBf78s/ls1hSFmkTdLXxcPPA14Tp21UH6mFuVebaoDSp82FkVFJ2uOucAJYfHa1s0bepwmbDAekm44V9rQ+xIYS3ucCen6ruzpxrpoosOCyz7UZDIwFop0E1frjVOOq3QC+S9TcY2rj/RsHsAWFhGsAG2qnk9CDGCl5p5A9BDDBBHfiGhURuvWL3BU7v1ytbf2G63pTadZ67x6679GT0EVe5vIpdWShMBPmrCyrqzFbi9uC/7Df9B1dyVrJaFlhjs984UjGV4C15Sgau3gFpyxfQg5MNh61RqplvxLohhJ4a/nYNsTa8erxVUOrRyr+fiU8YDQs6h2ohXfariLEsXqVLOd4y2FnMEotNGBz1vb107Xn4n57q8xpSW0i+2xB69vqxtiQMjq+taibVpDZ+MBeXT+J62pA7EKVlLyz5ijVt1/ZAX7oM+e1ZgZ/MUuhNkcbygOG7a+U4qKx

In [4]:
import os
from dotenv import load_dotenv

from langchain_google_genai import ChatGoogleGenerativeAI

llm_agent2 = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0
)
agent2 = create_agent(
    llm_agent2,
    tools=tools,
    system_prompt=(
        "You are Agent 2: The Clinical Reviewer. "
        "Critically review inputs, rank safest options, "
        "and warn about risks using evidence-based reasoning."
    )
)
response2 = agent2.invoke({
    "messages": [
        ("user", f"Review these recipes and select the safest ones:\n{agent1_output}")
    ]
})

print("===== Agent 2 Output (Test) =====")
print(response2["messages"][-1].content)



===== Agent 2 Output (Test) =====
[{'type': 'text', 'text': 'As a Clinical Reviewer, I have critically reviewed the provided recipes for blood sugar stability and general safety.\n\nHere are the recipes ranked from safest to least safe, along with evidence-based reasoning and warnings:\n\n**Safest Options:**\n\n1.  **Turkey and Quinoa Bowls with Roasted Vegetables and Avocado:**\n    *   **Reasoning:** This recipe is inherently well-balanced for blood sugar stability. Turkey provides lean protein, quinoa is a complex carbohydrate rich in fiber, roasted vegetables add essential vitamins, minerals, and fiber, and avocado contributes healthy monounsaturated fats. This combination helps slow glucose absorption, promote satiety, and minimize blood sugar spikes. Roasting is a generally healthy cooking method.\n    *   **Safety Score:** 5/5\n\n2.  **Turkey Chili:**\n    *   **Reasoning:** Turkey is a lean protein source. Chili typically includes beans and various vegetables, which are excelle

In [5]:
import os
from dotenv import load_dotenv

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tools import Tool
from langchain.agents import create_agent


# Step 1: Agent 1
response1 = agent1.invoke({
    "messages": [
        ("user", "List 5 recipes good for blood sugar stability.")
    ]
})

agent1_output = response1["messages"][-1].content

# Step 2: Pass Agent 1 output to Agent 2
response2 = agent2.invoke({
    "messages": [
        ("user", f"Audit and rank the following recipes:\n{agent1_output}")
    ]
})

print("===== FINAL MULTI-AGENT OUTPUT =====")
print(response2["messages"][-1].content)



===== FINAL MULTI-AGENT OUTPUT =====
[{'type': 'text', 'text': 'Here is a critical review and ranking of the provided recipes, focusing on blood sugar stability:\n\n**Critical Review and Ranking for Blood Sugar Stability:**\n\n1.  **Sweet Onion and Ham Frittata** (Rank: 1 - Safest)\n    *   **Reasoning:** This is likely the safest option for blood sugar stability. As a low-carb dish, a well-prepared frittata (with minimal or no added sugar to the onions, and plenty of non-starchy vegetables) will have a minimal impact on blood glucose. The protein from the eggs and ham will promote satiety and help stabilize blood sugar.\n    *   **Warning/Consideration:** Ensure no sugar is added when caramelizing or sweetening the onions, and opt for lean ham to reduce saturated fat intake. Incorporate a generous amount of other non-starchy vegetables (e.g., spinach, bell peppers).\n\n2.  **Homemade Yogurt** (Rank: 2)\n    *   **Reasoning:** The primary benefit here is "control over sugar content." U